# Script for master's thesis

This script contains the code for all the models included in the thesis. 

The script consist of the following: 
- Loading all the packages
- Importing data
- Defining ROC plot
- Logistsic regression 
    - Standard logistic regression
    - Ridge regression
    - Lasso regression
    - ROC plot for the three regression models
- Random forest
    - Variable importance
- Neural network
    - Definding the two neural networks
    - Early stopping
    - Neural network model 1
    - Neural network model 2
    - ROC plot for both neural networks


## Loading all the packages 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import animation, cm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from scipy import stats, optimize

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

seed_number = 1000
print_flag = 1
plot_flag = 1

## Importing data 

In [ ]:
# Importing the dataset
df=pd.read_excel(r'C:\Users\cullu\OneDrive - CBS - Copenhagen Business School\Speciale\Output_final.xlsx')
print(df)

# Removing the columns not needed from the dataset
useless = ["recordID","start_date", "end_date", "postal_code","premium","value","brand", "age_group", "seniority_group", "hp_group", "fuel_group"]
df_without_cats = df.drop(useless, axis=1)

# Adding dummies for 'age', 'seniority', 'hp', and 'fuel'
dummy_df=pd.get_dummies(df_without_cats,columns=["age","seniority","hp", "fuel"])
#print(dummy_df)


# Splitting dataset into two sets, X for the predictors and y for the response variable                                
X = dummy_df.drop(["status"], axis = 1)
y = dummy_df["status"]

# Splitting X and y into training dataset and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed_number)


## Defining ROC plot

In [ ]:
def roc_plot(values, colors, h=10, w=7, lab=None):
    sns.set()
    fig, ax = plt.subplots(figsize=(h, w))
    
    colors = colors
    i=0
    for val in values:
        ax.plot(val[1], val[0], label = '{} - AUC = {}'.format(lab[i], val[2]), color=colors[i])
        i += 1
        
    ax.plot(np.linspace(0, 1, 30), np.linspace(0, 1, 30), label='baseline', linestyle='--', color='orange')
    plt.title('Receiver Operating Characteristic', fontsize=18)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylim(0, 1.05)
    plt.legend(loc = 'lower right', fontsize=12)
    
    return values, colors

## Logistic regression

For logistsic regression we have computed three models: 
1. Standard logistic regression
2. Ridge regression
3. Lasso regression

### Standard logistic regression

In [ ]:
# Creating a standard logistic regression model
LR_stand = LogisticRegression()

# Gridsearch is used to find the optimal hyperparameters
parameters = {
    'solver': ['newton-cg', 'lbfgs', 'sag' and 'saga'],
    'penalty': ['none']
            }
LR_stand_Grid = GridSearchCV(                 LR_stand, 
                                         parameters, 
                                         cv=5, 
                                         scoring = "roc_auc", 
                                         # verbose=10, 
                                         n_jobs = -1
                                         )

# Training the standard logistic model
LR_stand_Grid.fit(X_train, y_train)
print(sorted(LR_stand_Grid.cv_results_.keys()))
LR_stand = LR_stand_Grid

# Printing the optimal hyperparameters
print(LR_stand.best_estimator_)
    
# Model prediction
LR_stand_preds = LR_stand.predict(X_test)

# Probability predicting for ROC plot
LR_stand_probs = LR_stand.predict_proba(X_test)[:,1]

# Model prediction with threshold
LR_stand_preds = np.where(LR_stand_probs > 0.35, 1, 0)

# Creating confusion matrix
y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = LR_stand_preds

confusion_matrix = pd.crosstab(y_prediction['status'], y_prediction["Preds"], rownames=['Actuals'], colnames=['Predicted'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrix)

# Performance measurements
performance_measurements = classification_report(LR_stand_preds, y_test ,output_dict = True)
if print_flag == 1: print("Performance measurements: \n",pd.DataFrame(performance_measurements))

# Creating ROC plot
if plot_flag == 1:
    fpr, tpr, threshold = metrics.roc_curve(y_test, LR_stand_probs)
    roc_auc = round(metrics.auc(fpr, tpr), 10)
    rates_list = [[tpr, fpr, roc_auc]]
    ## Creates the ROC plot
    roc_plot(values = rates_list, colors=['blue'], lab = ["Standard Logistic Regression"])
os.system('say "Standard logistic regression is done."')

### Ridge regression

In [ ]:
# Creating a ridge regression model
LR_ridge = LogisticRegression()

# Gridsearch is used to find the optimal hyperparameters
parameters = {
    'C': np.logspace(-3,3,50),
    'solver': ['newton-cg', 'lbfgs', 'sag' and 'saga'],
    'penalty': ['l2']
            }
LR_ridge_Grid = GridSearchCV(            LR_ridge, 
                                         parameters, 
                                         cv=5, 
                                         scoring = "roc_auc", 
                                         # verbose=10, 
                                         n_jobs = -1
                                         )

# Training the standard logistic model
LR_ridge_Grid.fit(X_train, y_train)
print(sorted(LR_ridge_Grid.cv_results_.keys()))
LR_ridge = LR_ridge_Grid

# Printing the optimal hyperparameters
print(LR_ridge.best_estimator_)
    
# Model prediction
LR_ridge_preds = LR_ridge.predict(X_test)

# Probability predicting for ROC plot
LR_ridge_probs = LR_ridge.predict_proba(X_test)[:,1]

# Model prediction with threshold
LR_ridge_preds = np.where(LR_ridge_probs > 0.35, 1, 0)

# Creating confusion matrix
y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = LR_ridge_preds

confusion_matrix = pd.crosstab(y_prediction['status'], y_prediction["Preds"], rownames=['Actuals'], colnames=['Predicted'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrix)

# Performance measurements
performance_measurements = classification_report(LR_ridge_preds, y_test, output_dict = True)
if print_flag == 1: print("Performance measurements: \n",pd.DataFrame(performance_measurements))

# Creating ROC plot
if plot_flag == 1:
    fpr, tpr, threshold = metrics.roc_curve(y_test, LR_ridge_probs)
    roc_auc = round(metrics.auc(fpr, tpr), 10)
    rates_list = [[tpr, fpr, roc_auc]]
    ## Creates the ROC plot
    roc_plot(values = rates_list, colors=['blue'], lab = ["Ridge Regression"])
os.system('say "Ridge regression is done."')

### Lasso regression

In [ ]:
# Creating a ridge regression model
LR_lasso = LogisticRegression()

# Gridsearch is used to find the optimal hyperparameters
parameters = {
    'C':np.arange(0,10,0.1),
    'solver': ['newton-cg', 'lbfgs', 'liblinear','sag' and 'saga'],
    'penalty' : ['l1']
            }
LR_lasso_Grid = GridSearchCV(            LR_lasso, 
                                         parameters, 
                                         cv=5, 
                                         scoring = "roc_auc", 
                                         # verbose=10, 
                                         n_jobs = -1
                                         )

# Training the standard logistic model
LR_lasso_Grid.fit(X_train, y_train)
print(sorted(LR_lasso_Grid.cv_results_.keys()))
LR_lasso = LR_lasso_Grid

# Printing the optimal hyperparameters
print(LR_lasso.best_estimator_)
    
# Model prediction
LR_lasso_preds = LR_lasso.predict(X_test)

# Probability predicting for ROC plot
LR_lasso_probs = LR_lasso.predict_proba(X_test)[:,1]

# Model prediction with threshold
LR_lasso_preds = np.where(LR_lasso_probs > 0.35, 1, 0)

# Creating confusion matrix
y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = LR_lasso_preds

confusion_matrix = pd.crosstab(y_prediction['status'], y_prediction["Preds"], rownames=['Actuals'], colnames=['Predicted'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrix)

# Performance measurements
performance_measurements = classification_report(LR_lasso_preds, y_test, output_dict = True)
if print_flag == 1: print("Performance measurements: \n",pd.DataFrame(performance_measurements))

# Creating ROC plot
if plot_flag == 1:
    fpr, tpr, threshold = metrics.roc_curve(y_test, LR_lasso_probs)
    roc_auc = round(metrics.auc(fpr, tpr), 10)
    rates_list = [[tpr, fpr, roc_auc]]
    ## Creates the ROC plot
    roc_plot(values = rates_list, colors=['blue'], lab = ["Lasso Regression"])
os.system('say "Lasso regression is done."')

### ROC plot for the three regression models

In [ ]:
# Calculating fpr and tpr for standard logistic regression
fpr_LR_stand, tpr_LR_stand, threshold_LR_stand = metrics.roc_curve(y_test, LR_stand_probs)
roc_auc_LR_stand = round(metrics.auc(fpr_LR_stand, tpr_LR_stand), 4)
rates_list_LR_stand = [[tpr_LR_stand, fpr_LR_stand, roc_auc_LR_stand]]

# Calculating fpr and tpr for ridge regression
fpr_LR_ridge, tpr_LR_ridge, threshold_LR_ridge = metrics.roc_curve(y_test, LR_ridge_probs)
roc_auc_LR_ridge = round(metrics.auc(fpr_LR_ridge, tpr_LR_ridge), 4)
rates_list_LR_ridge = [[tpr_LR_ridge, fpr_LR_ridge, roc_auc_LR_ridge]]

# Calculating fpr and tpr for lasso regression
fpr_LR_lasso, tpr_LR_lasso, threshold_LR_lasso = metrics.roc_curve(y_test, LR_lasso_probs)
roc_auc_LR_lasso = round(metrics.auc(fpr_LR_lasso, tpr_LR_lasso), 4)
rates_list_LR_lasso = [[tpr_LR_lasso, fpr_LR_lasso, roc_auc_LR_lasso]]

# Creating the ROC plot for standard logistic regression, lasso regression and ridge regression
rates_list = np.array([rates_list_LR_stand, rates_list_LR_ridge, rates_list_LR_lasso])
rates_list = rates_list.reshape(3,3)
colors_list = ['black'] + ['red'] + ['blue']
label_list = ["Standard Logistic Regression"] + ["Ridge Regression"] + ["Lasso Regression"]

roc_plot(values = rates_list, colors=colors_list, lab = label_list)

## Random forest 

In [ ]:
# Creating s random forest model
RF = RandomForestClassifier(  random_state = seed_number,
                                    bootstrap = True,
                                    oob_score = True,
                                    )

# Gridsearch is used to find the optimal hyperparameters (only the optimal hyperparameters are shown in the gridsearch, many different values have been tested)
parameters = {
     'criterion': ['entropy'],
     'max_depth':[10],
     'max_features': [8],
     'n_estimators': [1200],
             }
RF_Grid = GridSearchCV(          RF, 
                                 parameters, 
                                 cv=5, scoring = "roc_auc", 
                                 verbose=10, 
                                 # n_jobs = -1
                                         )

# Training the standard logistic model
RF_Grid.fit(X_train, y_train)
print(sorted(RF.cv_results_.keys()))
RF = RF_Grid

# Printing the optimal hyperparameters
print(RF.best_estimator_)
    
# Model prediction
RF_preds = RF.predict(X_test)

# Probability predicting for ROC plot
RF_probs = RF.predict_proba(X_test)[:,1]

# Model prediction with threshold
RF_preds = np.where(RF_probs > 0.35, 1, 0)

# Creating confusion matrix
y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = RF_preds

confusion_matrix = pd.crosstab(y_prediction['status'], y_prediction["Preds"], rownames=['Actuals'], colnames=['Predicted'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrix)

# Performance measurements
performance_measurements = classification_report(RF_preds, y_test, output_dict = True)
if print_flag == 1: print("Performance measurements: \n",pd.DataFrame(performance_measurements))

# Creating ROC plot
if plot_flag == 1:
    fpr, tpr, threshold = metrics.roc_curve(y_test, RF_probs)
    roc_auc = round(metrics.auc(fpr, tpr), 10)
    rates_list = [[tpr, fpr, roc_auc]]
    ## Creates the ROC plot
    roc_plot(values = rates_list, colors=['blue'], lab = ["Random forest"])
os.system('say "Random forest is done."')

### Variable importance

In [ ]:
# Printing variable importance for the random forest model with the optimal hyperparameters
RF = RandomForestClassifier(  max_depth = 10, 
                                    random_state = seed_number,
                                    bootstrap = True,
                                    criterion = 'entropy', 
                                    max_features = 8,
                                    # max_features = "sqrt",
                                    n_estimators = 1200,
                                    oob_score = True
                                    #class_weight = "balanced"
                                )
RF.fit(X_train, y_train)

RF.feature_importances_

RF.oob_score_

import numpy as np
 
importances = RF.feature_importances_
#
# Sort the feature importance in descending order
#
sorted_indices = np.argsort(importances)[::-1]
 
feat_labels = dummy_df.columns[1:]
 
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,
                            feat_labels[sorted_indices[f]],
                            importances[sorted_indices[f]]))

## Neural network

### Definding the two neural networks

In [ ]:
#Defines neural network model 1
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Dense(8, input_dim=X_train.shape[1], activation='relu'))
model1.add(tf.keras.layers.Dense(6, activation='relu'))
model1.add(tf.keras.layers.Dense(4, activation='relu'))
model1.add(tf.keras.layers.Dense(2, activation='relu'))
model1.add(tf.keras.layers.Dense(1, activation='sigmoid'))

#Defines neural network model 2
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Dense(20, input_dim=X_train.shape[1], activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Early stopping

In [ ]:
#compiles the two models
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


mc1 = tf.keras.callbacks.ModelCheckpoint('model1.h5', monitor='loss', mode='min', verbose=1, save_best_only=True)
mc2 = tf.keras.callbacks.ModelCheckpoint('model2.h5', monitor='loss', mode='min', verbose=1, save_best_only=True)


#transposes the y_train set
y_train = np.array(y_train)
y_train = y_train.T

#defines early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

#make early stopping for neural network model 1
EarlyStopping_model_1 = model1.fit(X_train, y_train, epochs=1000, batch_size=10, callbacks=[es, mc1], validation_split=0.2)

#print the training loss and validation loss plot for neural network model 1
fig, ax = plt.subplots(figsize=(10, 10))

pyplot.plot(EarlyStopping_model_1.history['loss'], label='train')
pyplot.plot(EarlyStopping_model_1.history['val_loss'], label='validation error')
pyplot.legend()
pyplot.show()

#make early stopping for neural network model 2
EarlyStopping_model_2 = model2.fit(X_train, y_train, epochs=1000, batch_size=10, callbacks=[es, mc2], validation_split=0.2)

#print the training loss and validation loss plot for neural network model 2
fig, ax = plt.subplots(figsize=(10, 10))

pyplot.plot(EarlyStopping_model_2.history['loss'], label='train')
pyplot.plot(EarlyStopping_model_2.history['val_loss'], label='validation error')
pyplot.legend()
pyplot.show()

### Neural network model 1

In [ ]:
#We have now find the optimal number of epochs and can now set up the two models with optimal parameter:

#Neural network model 1
def create_NN_model1():
    n_weight = 1
    end_nodes = 4
    NN_model1 = tf.keras.Sequential()
    NN_model1.add(Dense(8, activation='relu', input_shape=(24,)))
    NN_model1.add(Dense(6, activation='relu'))
    NN_model1.add(Dense(4, activation='relu'))
    NN_model1.add(Dense(2, activation='relu'))
    NN_model1.add(Dense(1, activation='sigmoid'))
    #activation = sigmoid for the output layer as we have a binary classification problem 

    #defines the model loss function, opimizer og goal of perfomance
    sgd = SGD(lr=0.01, momentum=0.9)
    NN_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return NN_model1

#Splits the dataset into X set with the predictors and y sets with the response variable                               
X = dummy_df.drop(["status"], axis = 1)
y = dummy_df["status"]

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

scaler = StandardScaler() 

#Make the test and train datasets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.2, random_state=1000)


#run the model with the optimal hyperparameters found
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_NN_model1, epochs=150, batch_size=10, verbose=10)))
NN_model1 = Pipeline(estimators)

#fits the model
NN_model_1 = NN_model1.fit(X_train, y_train)

NN_Model_preds1 = NN_model_1.predict(X_test)


NN_Model_probs1 = NN_model_1.predict_proba(X_test)[:,1]
predictions1 = np.where(FF_Model_probs1 > 0.5, 1, 0)

y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = NN_Model_preds1

print(y_prediction)

#Model prediction
NN_model_probs1 = NN_model_1.predict_proba(X_test)[:,1]
NN_Model_preds1 = np.where(NN_model_probs1 > 0.5, 1, 0)

#Used for confusion matrix
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = NN_Model_preds1

#Make confusion matrix
confusion_matrixs = pd.crosstab(y_prediction['Preds'], y_prediction["status"], rownames=['Predicted'], colnames=['Actual'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrixs)


# Performance measurements
performance_measurements = classification_report(y_test, y_prediction["Preds"],output_dict = True)
if print_flag == 1: print("Performance measurements: \n",pd.DataFrame(performance_measurements))

    
#Probability predicting for ROC plot
FF_Model_probs = NN_model_1.predict_proba(X_test)[:,1]


#ROC plot
if plot_flag == 1:
    fpr1, tpr1, threshold1 = metrics.roc_curve(y_test, NN_model_probs1)
    roc_auc1 = round(metrics.auc(fpr1, tpr1), 10)
    rates_list1 = [[tpr1, fpr1, roc_auc1]]
    
    #Creates the ROC plot
    roc_plot(values = rates_list1, colors=['blue'], lab = ["Neural Network model 1"])

### Neural network model 2

In [ ]:
#Neural network model 2

def create_NN_model2():
    # Sequential bygger modellen
    n_weight = 1
    end_nodes = 4
    NN_model2 = tf.keras.Sequential()
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(20, activation='relu'))
    NN_model2.add(Dropout(0.2))
    NN_model2.add(Dense(1, activation='sigmoid'))
    #activation = sigmoid for the output layer as we have a binary classification problem 

    #defines the model loss function, opimizer og goal of perfomance
    sgd = SGD(lr=0.01, momentum=0.9)
    NN_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
  
    return NN_model2

#Splits the dataset into X set with the predictors and y sets with the response variable                               
X = dummy_df.drop(["status"], axis = 1)
y = dummy_df["status"]

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

scaler = StandardScaler() 

#Make the test and train datasets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.2, random_state=1000)


#run the model with the optimal hyperparameters found
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_NN_model2, epochs=133, batch_size=32, verbose=10)))
NN_model2 = Pipeline(estimators)


#fits the model
NN_model_2 = NN_model2.fit(X_train, y_train)

NN_Model_preds2 = NN_model_2.predict(X_test)


NN_Model_probs2 = NN_model_2.predict_proba(X_test)[:,1]
predictions2 = np.where(NN_Model_probs2 > 0.5, 1, 0)

y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = NN_Model_preds2

#Model prediction
NN_model_2_probs = NN_model_2.predict_proba(X_test)[:,1]
NN_model_2_preds = np.where(NN_model_2_probs > 0.5, 1, 0)


#Used for confusion matrix
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = NN_model_2_preds

#Makes confusion matrix
confusion_matrixs = pd.crosstab(y_prediction['Preds'], y_prediction["status"], rownames=['Predicted'], colnames=['Actual'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrixs)

# Performance measurements
performance_measurements = classification_report(y_test, y_prediction["Preds"],output_dict = True)
if print_flag == 1: print("Performance measurements: \n",pd.DataFrame(performance_measurements))
    
#Probability predicting for ROC plot
NN_model_2_probs = NN_model_2.predict_proba(X_test)[:,1]


#ROC plot
if plot_flag == 1:
    fpr2, tpr2, threshold2 = metrics.roc_curve(y_test, NN_model_2_probs)
    roc_auc2 = round(metrics.auc(fpr2, tpr2), 10)
    rates_list2 = [[tpr2, fpr2, roc_auc2]]
    
    #Creates the ROC plot
    roc_plot(values = rates_list2, colors=['blue'], lab = ["Neural Network model 2"])

### ROC plot for both neural networks

In [ ]:
#Compare the ROC and AUC for the two models

#Neural network model 1
fpr1, tpr1, threshold1 = metrics.roc_curve(y_test, NN_model_probs1)
roc_auc1 = round(metrics.auc(fpr1, tpr1), 4)
rates_list1 = [[tpr1, fpr1, roc_auc1]]

#Neural network model 2
fpr2, tpr2, threshold2 = metrics.roc_curve(y_test, NN_model_2_probs)
roc_auc2 = round(metrics.auc(fpr2, tpr2), 4)
rates_list2 = [[tpr2, fpr2, roc_auc2]]


#The combined ROC plot
rates_list_combined = [[tpr1, fpr1, roc_auc1], [tpr2, fpr2, roc_auc2]]
colors_list = ['blue'] + ['red']
label_list = ["Neural Network model 1"] + ["Neural Network model 2"]

#Creates the ROC plot
roc_plot(values = rates_list_combined, colors=colors_list, lab = label_list)